In [1]:
import os
import json
import pandas as pd

In [2]:
ds = json.load(open("../data/annotations/annotations/instances_train2017.json"))

In [3]:
ds.keys()

dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])

In [5]:
images = pd.DataFrame(ds['images'])

In [6]:
annotations = pd.DataFrame(ds['annotations'])

In [13]:
images.head()

,license,file_name,coco_url,height,width,date_captured,flickr_url,id
0,3,000000391895.jpg,http://images.cocodataset.org/train2017/000000...,360,640,2013-11-14 11:18:45,http://farm9.staticflickr.com/8186/8119368305_...,391895
1,4,000000522418.jpg,http://images.cocodataset.org/train2017/000000...,480,640,2013-11-14 11:38:44,http://farm1.staticflickr.com/1/127244861_ab0c...,522418
2,3,000000184613.jpg,http://images.cocodataset.org/train2017/000000...,336,500,2013-11-14 12:36:29,http://farm3.staticflickr.com/2169/2118578392_...,184613
3,3,000000318219.jpg,http://images.cocodataset.org/train2017/000000...,640,556,2013-11-14 13:02:53,http://farm5.staticflickr.com/4125/5094763076_...,318219
4,3,000000554625.jpg,http://images.cocodataset.org/train2017/000000...,640,426,2013-11-14 16:03:19,http://farm5.staticflickr.com/4086/5094162993_...,554625


In [15]:
images = images[['file_name', 'id']]

In [16]:
annotations = annotations[['image_id', 'bbox', 'category_id']]

In [17]:
images.columns, annotations.columns

(Index(['file_name', 'id'], dtype='object'),
 Index(['image_id', 'bbox', 'category_id'], dtype='object'))

In [19]:
merge_df = pd.merge(images, annotations, how='inner', left_on='id', right_on='image_id')

In [21]:
del merge_df['id']

In [22]:
merge_df.head()

,file_name,image_id,bbox,category_id
0,000000391895.jpg,391895,"[359.17, 146.17, 112.45, 213.57]",4
1,000000391895.jpg,391895,"[339.88, 22.16, 153.88, 300.73]",1
2,000000391895.jpg,391895,"[471.64, 172.82, 35.92, 48.1]",1
3,000000391895.jpg,391895,"[486.01, 183.31, 30.63, 34.98]",2
4,000000522418.jpg,522418,"[382.48, 0.0, 256.8, 474.31]",1


In [24]:
def helper(g_df):
    f = list(zip(g_df['bbox'].tolist(), g_df['category_id'].tolist()))
    return f

f_df = merge_df.groupby(['file_name', 'image_id']).apply(helper).reset_index()

In [28]:
f_df = f_df.rename(columns={0:"box_category"})

In [33]:
f_df.iloc[0].to_dict()

{'file_name': '000000000009.jpg',
 'image_id': 9,
 'box_category': [([1.08, 187.69, 611.59, 285.84], 51),
  ([311.73, 4.31, 319.28, 228.68], 51),
  ([249.6, 229.27, 316.24, 245.08], 56),
  ([0.0, 13.51, 434.48, 375.12], 51),
  ([376.2, 40.36, 75.55, 46.53], 55),
  ([465.78, 38.97, 58.07, 46.67], 55),
  ([385.7, 73.66, 84.02, 70.51], 55),
  ([364.05, 2.49, 94.76, 71.07], 55)]}

In [32]:
from tqdm import tqdm
ANNOTATION_PATH = ""

for i, row in merge_df.iterrows():
    filename = row['file_name'].replace(".jpg",'.json')
    json.dump(open(f"{ANNOTATION_PATH}/{filename}"))

([311.73, 4.31, 319.28, 228.68], 51)